In [9]:
from typing import Sequence

import clingo

In [10]:
def solve(programs: Sequence[str], grounding_context=None, sep:str=' '):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            symbols = model.symbols(atoms=True)
            print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        mode = "UNKNOWN"
        solve_result = solve_handle.get()
        if solve_result.satisfiable:
            mode = "SAT"
        elif solve_result.unsatisfiable:
            mode = "UNSAT"
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(mode, "{}{}".format(len(models), cardinality_suffix))

    return models

In [11]:
agents = """

agent(alice).
agent(bob).

"""
solve([agents]);

Answer 1: { agent(alice) agent(bob) }
SAT 1


In [12]:
commodities = """

commodity("EUR").

"""
solve([commodities]);

Answer 1: { commodity("EUR") }
SAT 1


In [13]:
time = """

time(0..3).
startTime(ST) :- time(ST), not time(ST - 1).
endTime(ET) :- time(ET), not time(ET + 1).

"""
solve([time]);

Answer 1: { endTime(3) startTime(0) time(0) time(1) time(2) time(3) }
SAT 1


In [14]:
accounts = """

agent_account_commodity_type(Agent, "Aktiva:Kassa:EUR", "EUR", real) :- agent(Agent).
agent_account_commodity_type(Agent, "Passiva:Eigenkapital:EUR", "EUR", logical) :- agent(Agent).
agent_account_commodity_type(Agent, "Passiva:Verbindlichkeiten:EUR", "EUR", logical) :- agent(Agent).

agent_equityAccount_commodity(Agent, "Passiva:Eigenkapital:EUR", "EUR") :- agent(Agent).

"""
solve([accounts]);

Answer 1: {  }
SAT 1


<block>:3:73-85: info: atom does not occur in any rule head:
  agent(Agent)

<block>:4:84-96: info: atom does not occur in any rule head:
  agent(Agent)

<block>:5:89-101: info: atom does not occur in any rule head:
  agent(Agent)

<block>:7:76-88: info: atom does not occur in any rule head:
  agent(Agent)



In [15]:
init = """

obs_at(agent_account_debits_credits_commodity(Agent, Account, 0, 0, Commodity), ST) :-
 startTime(ST),
 agent_account_commodity_type(Agent, Account, Commodity, _Type).

occ_at(agent_debitAccount_agent_creditAccount_amount_commodity(Agent, Account, Agent, EquityAccount, InitialBalance, Commodity), ST) :-
  startTime(ST),
  agent_account_initialBalance_commodity(Agent, Account, InitialBalance, Commodity),
  agent_equityAccount_commodity(Agent, EquityAccount, Commodity).

"""
solve([init]);

Answer 1: {  }
SAT 1


<block>:4:2-15: info: atom does not occur in any rule head:
  startTime(ST)

<block>:5:2-64: info: atom does not occur in any rule head:
  agent_account_commodity_type(Agent,Account,Commodity,_Type)

<block>:8:3-16: info: atom does not occur in any rule head:
  startTime(ST)

<block>:9:3-84: info: atom does not occur in any rule head:
  agent_account_initialBalance_commodity(Agent,Account,InitialBalance,Commodity)

<block>:10:3-65: info: atom does not occur in any rule head:
  agent_equityAccount_commodity(Agent,EquityAccount,Commodity)



In [16]:
transaction = """

obs_at(agent_account_debitChange_commodity(Agent, Account, Change, Commodity), T) :-
  time(T),
  agent_account_commodity_type(Agent, Account, Commodity, _Type),
  Change = #sum{ Amount,CreditAgent,CreditAccount : occ_at(agent_debitAccount_agent_creditAccount_amount_commodity(Agent, Account, CreditAgent, CreditAccount, Amount, Commodity), T) }.

obs_at(agent_account_creditChange_commodity(Agent, Account, Change, Commodity), T) :-
  time(T),
  agent_account_commodity_type(Agent, Account, Commodity, _Type),
  Change = #sum{ Amount,DebitAgent,DebitAccount : occ_at(agent_debitAccount_agent_creditAccount_amount_commodity(DebitAgent, DebitAccount, Agent, Account, Amount, Commodity), T) }.

obs_at(agent_account_debits_credits_commodity(Agent, Account, PreviousDebits + Debits, PreviousCredits + Credits, Commodity), T+1) :-
  time(T+1),
  agent_account_commodity_type(Agent, Account, Commodity, _Type),
  obs_at(agent_account_debits_credits_commodity(Agent, Account, PreviousDebits, PreviousCredits, Commodity), T),
  obs_at(agent_account_debitChange_commodity(Agent, Account, Debits, Commodity), T),
  obs_at(agent_account_creditChange_commodity(Agent, Account, Credits, Commodity), T).

"""
solve([transaction]);

Answer 1: {  }
SAT 1


<block>:4:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:5:3-65: info: atom does not occur in any rule head:
  agent_account_commodity_type(Agent,Account,Commodity,_Type)

<block>:6:53-182: info: atom does not occur in any rule head:
  occ_at(agent_debitAccount_agent_creditAccount_amount_commodity(Agent,Account,CreditAgent,CreditAccount,Amount,Commodity),T)

<block>:9:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:10:3-65: info: atom does not occur in any rule head:
  agent_account_commodity_type(Agent,Account,Commodity,_Type)

<block>:11:51-178: info: atom does not occur in any rule head:
  occ_at(agent_debitAccount_agent_creditAccount_amount_commodity(DebitAgent,DebitAccount,Agent,Account,Amount,Commodity),T)

<block>:14:3-12: info: atom does not occur in any rule head:
  time((T+1))

<block>:15:3-65: info: atom does not occur in any rule head:
  agent_account_commodity_type(Agent,Account,Commodity,_Type)



In [17]:
consistency = """


"""
solve([consistency]);

Answer 1: {  }
SAT 1


In [18]:
instance = """
agent_account_initialBalance_commodity(alice, "Aktiva:Kassa", 100, "EUR").

"""
solve([instance]);

Answer 1: { agent_account_initialBalance_commodity(alice,"Aktiva:Kassa",100,"EUR") }
SAT 1


In [19]:
solve([
    agents,
    commodities,
    time,
    accounts,
    init,
    transaction,
    consistency,
    instance,
], sep='\n');

Answer 1: {
agent(alice)
agent(bob)
commodity("EUR")
endTime(3)
startTime(0)
time(0)
time(1)
time(2)
time(3)
obs_at(agent_account_creditChange_commodity(alice,"Aktiva:Kassa:EUR",0,"EUR"),0)
obs_at(agent_account_creditChange_commodity(alice,"Aktiva:Kassa:EUR",0,"EUR"),1)
obs_at(agent_account_creditChange_commodity(alice,"Aktiva:Kassa:EUR",0,"EUR"),2)
obs_at(agent_account_creditChange_commodity(alice,"Aktiva:Kassa:EUR",0,"EUR"),3)
obs_at(agent_account_creditChange_commodity(alice,"Passiva:Eigenkapital:EUR",0,"EUR"),1)
obs_at(agent_account_creditChange_commodity(alice,"Passiva:Eigenkapital:EUR",0,"EUR"),2)
obs_at(agent_account_creditChange_commodity(alice,"Passiva:Eigenkapital:EUR",0,"EUR"),3)
obs_at(agent_account_creditChange_commodity(alice,"Passiva:Eigenkapital:EUR",100,"EUR"),0)
obs_at(agent_account_creditChange_commodity(alice,"Passiva:Verbindlichkeiten:EUR",0,"EUR"),0)
obs_at(agent_account_creditChange_commodity(alice,"Passiva:Verbindlichkeiten:EUR",0,"EUR"),1)
obs_at(agent_account_